In [1]:
from rrgcn import RRGCNEmbedder
from torch_geometric.datasets import Entities
import torch
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import rdflib

In [2]:
## edited entities.py from PyG that also saves node_features
import logging
import os
import os.path as osp
from collections import Counter
from typing import Callable, List, Optional

import numpy as np
import torch

from torch_geometric.data import (
    Data,
    InMemoryDataset,
    download_url,
    extract_tar,
)


class Entities(InMemoryDataset):
    url = 'https://data.dgl.ai/dataset/{}.tgz'

    def __init__(self, root: str, name: str,
                 transform: Optional[Callable] = None,
                 pre_transform: Optional[Callable] = None):
        self.name = name.lower()
        assert self.name in ['aifb', 'am', 'mutag', 'bgs']
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_dir(self) -> str:
        return osp.join(self.root, self.name, 'raw')

    @property
    def processed_dir(self) -> str:
        return osp.join(self.root, self.name, 'processed')

    @property
    def num_relations(self) -> int:
        return self.data.edge_type.max().item() + 1

    @property
    def num_classes(self) -> int:
        return self.data.train_y.max().item() + 1

    @property
    def raw_file_names(self) -> List[str]:
        return [
            f'{self.name}_stripped.nt.gz',
            'completeDataset.tsv',
            'trainingSet.tsv',
            'testSet.tsv',
        ]

    @property
    def processed_file_names(self) -> str:
        return 'data.pt'

    def download(self):
        path = download_url(self.url.format(self.name), self.root)
        extract_tar(path, self.raw_dir)
        os.unlink(path)

    def process(self):
        import gzip

        import pandas as pd
        import rdflib as rdf

        graph_file, task_file, train_file, test_file = self.raw_paths

        with hide_stdout():
            g = rdf.Graph()
            with gzip.open(graph_file, 'rb') as f:
                g.parse(file=f, format='nt')

        freq = Counter(g.predicates())

        relations = sorted(set(g.predicates()), key=lambda p: -freq.get(p, 0))
        subjects = set(g.subjects())
        objects = set(g.objects())
        nodes = list(subjects.union(objects))

        N = len(nodes)
        R = 2 * len(relations)

        relations_dict = {rel: i for i, rel in enumerate(relations)}
        nodes_dict = {node: i for i, node in enumerate(nodes)}

        edges = []
        node_features = {}
        for s, p, o in g.triples((None, None, None)):
            src, dst, rel = nodes_dict[s], nodes_dict[o], relations_dict[p]
            edges.append([src, dst, 2 * rel])
            edges.append([dst, src, 2 * rel + 1])

            # SAVE LITERAL VALUES
            if isinstance(o, rdf.Literal):
                literal_type = p#type(o.value)
                if literal_type not in node_features:
                    node_features[literal_type] = [[dst], [o.value]]
                else:
                    node_features[literal_type][0].append(dst)
                    node_features[literal_type][1].append(o.value)

        edges = torch.tensor(edges, dtype=torch.long).t().contiguous()
        perm = (N * R * edges[0] + R * edges[1] + edges[2]).argsort()
        edges = edges[:, perm]

        edge_index, edge_type = edges[:2], edges[2]

        if self.name == 'am':
            label_header = 'label_cateogory'
            nodes_header = 'proxy'
        elif self.name == 'aifb':
            label_header = 'label_affiliation'
            nodes_header = 'person'
        elif self.name == 'mutag':
            label_header = 'label_mutagenic'
            nodes_header = 'bond'
        elif self.name == 'bgs':
            label_header = 'label_lithogenesis'
            nodes_header = 'rock'

        labels_df = pd.read_csv(task_file, sep='\t')
        labels_set = set(labels_df[label_header].values.tolist())
        labels_dict = {lab: i for i, lab in enumerate(list(labels_set))}
        nodes_dict = {np.unicode(key): val for key, val in nodes_dict.items()}

        train_labels_df = pd.read_csv(train_file, sep='\t')
        train_indices, train_labels = [], []
        for nod, lab in zip(train_labels_df[nodes_header].values,
                            train_labels_df[label_header].values):
            train_indices.append(nodes_dict[nod])
            train_labels.append(labels_dict[lab])

        train_idx = torch.tensor(train_indices, dtype=torch.long)
        train_y = torch.tensor(train_labels, dtype=torch.long)

        test_labels_df = pd.read_csv(test_file, sep='\t')
        test_indices, test_labels = [], []
        for nod, lab in zip(test_labels_df[nodes_header].values,
                            test_labels_df[label_header].values):
            test_indices.append(nodes_dict[nod])
            test_labels.append(labels_dict[lab])

        test_idx = torch.tensor(test_indices, dtype=torch.long)
        test_y = torch.tensor(test_labels, dtype=torch.long)

        data = Data(edge_index=edge_index, edge_type=edge_type,
                    train_idx=train_idx, train_y=train_y, test_idx=test_idx,
                    test_y=test_y, num_nodes=N, node_features=node_features)

        torch.save(self.collate([data]), self.processed_paths[0])

    def __repr__(self) -> str:
        return f'{self.name.upper()}{self.__class__.__name__}()'


class hide_stdout(object):
    def __enter__(self):
        self.level = logging.getLogger().level
        logging.getLogger().setLevel(logging.ERROR)

    def __exit__(self, *args):
        logging.getLogger().setLevel(self.level)

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
dataset = Entities('./', 'mutag')
data = dataset[0].to(device)

In [4]:
charge = rdflib.URIRef("http://dl-learner.org/carcinogenesis#charge")
filtered_node_features = {}
filtered_node_features[0] = [None, None]
filtered_node_features[0][0] = torch.tensor(
    data.node_features[charge][0], dtype=torch.long
)
filtered_node_features[0][1] = torch.tensor(
    data.node_features[charge][1], dtype=torch.float32
).reshape(-1, 1)

In [5]:
train_idx, val_idx, y_train, y_val = train_test_split(
    data.train_idx,
    data.train_y,
    stratify=data.train_y.cpu().numpy(),
    test_size=0.1,
    random_state=42,
)

In [6]:
embedder = RRGCNEmbedder(
    num_nodes=data.num_nodes,
    num_relations=dataset.num_relations,
    num_layers=5,
    emb_size=2048,
    device=device,
)

In [7]:
# for node features to work well, they have to be normalized
# you can choose "standard" for StandardScaler, "robust" for RobustScaler, "quantile"
# for QuantileTransformer and "power" for PowerTransformer
#
# you could also pass sklearn compatible scalers by passing a dict keyed by
# literal type, e.g.: {0: sklearn.preprocessing.StandardScaler()}
train_embs = embedder.embeddings(
    data.edge_index,
    data.edge_type,
    node_features=filtered_node_features,
    node_features_scalers="robust",
    idx=train_idx,
)

100%|██████████| 1/1 [00:13<00:00, 13.18s/it]


In [8]:
# only fit node feature scalers on nodes reachable from train nodes,
# for val and test nodes, reuse the fit scalers using embedder.get_last_fit_scalers()
val_embs = embedder.embeddings(
    data.edge_index,
    data.edge_type,
    node_features=filtered_node_features,
    node_features_scalers=embedder.get_last_fit_scalers(),
    idx=val_idx,
)
test_embs = embedder.embeddings(
    data.edge_index,
    data.edge_type,
    node_features=filtered_node_features,
    node_features_scalers=embedder.get_last_fit_scalers(),
    idx=data.test_idx,
)

100%|██████████| 1/1 [00:13<00:00, 13.22s/it]


In [9]:
task_type = "GPU" if torch.cuda.is_available() else "CPU"
clf = CatBoostClassifier(
    iterations=1000,
    early_stopping_rounds=100,
    task_type=task_type,
    random_seed=42,
    use_best_model=True,
    auto_class_weights="Balanced",
)
clf = clf.fit(
    train_embs.cpu().numpy(),
    y_train.cpu().numpy(),
    eval_set=(val_embs.cpu().numpy(), y_val.cpu().numpy()),
)

Learning rate set to 0.080278
0:	learn: 0.6394734	test: 0.6480066	best: 0.6480066 (0)	total: 143ms	remaining: 2m 22s
1:	learn: 0.6043138	test: 0.6160162	best: 0.6160162 (1)	total: 281ms	remaining: 2m 20s
2:	learn: 0.5708737	test: 0.5830180	best: 0.5830180 (2)	total: 417ms	remaining: 2m 18s
3:	learn: 0.5516711	test: 0.5606688	best: 0.5606688 (3)	total: 549ms	remaining: 2m 16s
4:	learn: 0.5237342	test: 0.5517336	best: 0.5517336 (4)	total: 690ms	remaining: 2m 17s
5:	learn: 0.4953295	test: 0.5436822	best: 0.5436822 (5)	total: 835ms	remaining: 2m 18s
6:	learn: 0.4811546	test: 0.5335904	best: 0.5335904 (6)	total: 923ms	remaining: 2m 10s
7:	learn: 0.4684954	test: 0.5225592	best: 0.5225592 (7)	total: 1.06s	remaining: 2m 11s
8:	learn: 0.4389852	test: 0.4974404	best: 0.4974404 (8)	total: 1.21s	remaining: 2m 12s
9:	learn: 0.4252482	test: 0.5018680	best: 0.4974404 (8)	total: 1.35s	remaining: 2m 13s
10:	learn: 0.4090147	test: 0.4854347	best: 0.4854347 (10)	total: 1.48s	remaining: 2m 13s
11:	learn: 

In [10]:
print(classification_report(clf.predict(test_embs.cpu().numpy()), data.test_y.cpu().numpy()))

              precision    recall  f1-score   support

           0       0.96      0.88      0.91        49
           1       0.74      0.89      0.81        19

    accuracy                           0.88        68
   macro avg       0.85      0.89      0.86        68
weighted avg       0.90      0.88      0.89        68



In [11]:
# performance of same embedder where literals are encoded as regular nodes
train_embs = embedder.embeddings(
    data.edge_index,
    data.edge_type,
    idx=train_idx,
)
val_embs = embedder.embeddings(
    data.edge_index,
    data.edge_type,
    idx=val_idx,
)
test_embs = embedder.embeddings(
    data.edge_index,
    data.edge_type,
    idx=data.test_idx,
)
task_type = "GPU" if torch.cuda.is_available() else "CPU"
clf = CatBoostClassifier(
    iterations=1000,
    early_stopping_rounds=100,
    task_type=task_type,
    random_seed=42,
    use_best_model=True,
    auto_class_weights="Balanced",
)
clf = clf.fit(
    train_embs.cpu().numpy(),
    y_train.cpu().numpy(),
    eval_set=(val_embs.cpu().numpy(), y_val.cpu().numpy()),
)
print(classification_report(clf.predict(test_embs.cpu().numpy()), data.test_y.cpu().numpy()))

100%|██████████| 1/1 [00:13<00:00, 13.30s/it]


Learning rate set to 0.080278
0:	learn: 0.6538378	test: 0.6605008	best: 0.6605008 (0)	total: 142ms	remaining: 2m 22s
1:	learn: 0.6171080	test: 0.6224138	best: 0.6224138 (1)	total: 283ms	remaining: 2m 21s
2:	learn: 0.5819143	test: 0.6069669	best: 0.6069669 (2)	total: 418ms	remaining: 2m 18s
3:	learn: 0.5595998	test: 0.5866527	best: 0.5866527 (3)	total: 555ms	remaining: 2m 18s
4:	learn: 0.5399098	test: 0.5812235	best: 0.5812235 (4)	total: 699ms	remaining: 2m 19s
5:	learn: 0.5284173	test: 0.5648007	best: 0.5648007 (5)	total: 839ms	remaining: 2m 19s
6:	learn: 0.5058470	test: 0.5629640	best: 0.5629640 (6)	total: 979ms	remaining: 2m 18s
7:	learn: 0.4844777	test: 0.5601503	best: 0.5601503 (7)	total: 1.12s	remaining: 2m 18s
8:	learn: 0.4593012	test: 0.5369273	best: 0.5369273 (8)	total: 1.27s	remaining: 2m 19s
9:	learn: 0.4459104	test: 0.5212256	best: 0.5212256 (9)	total: 1.41s	remaining: 2m 19s
10:	learn: 0.4331912	test: 0.5184409	best: 0.5184409 (10)	total: 1.56s	remaining: 2m 19s
11:	learn: 

In [13]:
# performance of same embedder where literals are removed from the graph
literals = filtered_node_features[0][0].squeeze().to(device)
mask = torch.isin(data.edge_index[0], literals) | torch.isin(data.edge_index[1], literals)
edge_index = data.edge_index[:, ~mask]
edge_type = data.edge_type[~mask]

train_embs = embedder.embeddings(
    edge_index,
    edge_type,
    idx=train_idx,
)
val_embs = embedder.embeddings(
    edge_index,
    edge_type,
    idx=val_idx,
)
test_embs = embedder.embeddings(
    edge_index,
    edge_type,
    idx=data.test_idx,
)
task_type = "GPU" if torch.cuda.is_available() else "CPU"
clf = CatBoostClassifier(
    iterations=1000,
    early_stopping_rounds=100,
    task_type=task_type,
    random_seed=42,
    use_best_model=True,
    auto_class_weights="Balanced",
)
clf = clf.fit(
    train_embs.cpu().numpy(),
    y_train.cpu().numpy(),
    eval_set=(val_embs.cpu().numpy(), y_val.cpu().numpy()),
)
print(classification_report(clf.predict(test_embs.cpu().numpy()), data.test_y.cpu().numpy()))

100%|██████████| 1/1 [00:12<00:00, 12.58s/it]


Learning rate set to 0.080278
0:	learn: 0.6609449	test: 0.6597681	best: 0.6597681 (0)	total: 146ms	remaining: 2m 25s
1:	learn: 0.6357253	test: 0.6465873	best: 0.6465873 (1)	total: 282ms	remaining: 2m 20s
2:	learn: 0.6027870	test: 0.6190348	best: 0.6190348 (2)	total: 419ms	remaining: 2m 19s
3:	learn: 0.5868718	test: 0.6099773	best: 0.6099773 (3)	total: 555ms	remaining: 2m 18s
4:	learn: 0.5571536	test: 0.6033397	best: 0.6033397 (4)	total: 699ms	remaining: 2m 19s
5:	learn: 0.5379230	test: 0.5891080	best: 0.5891080 (5)	total: 845ms	remaining: 2m 20s
6:	learn: 0.5200257	test: 0.5608741	best: 0.5608741 (6)	total: 981ms	remaining: 2m 19s
7:	learn: 0.5018957	test: 0.5454596	best: 0.5454596 (7)	total: 1.13s	remaining: 2m 19s
8:	learn: 0.4798202	test: 0.5418535	best: 0.5418535 (8)	total: 1.26s	remaining: 2m 19s
9:	learn: 0.4711153	test: 0.5369572	best: 0.5369572 (9)	total: 1.4s	remaining: 2m 18s
10:	learn: 0.4580398	test: 0.5212490	best: 0.5212490 (10)	total: 1.54s	remaining: 2m 18s
11:	learn: 0